In [ ]:
# pip install statsmodels

import numpy as np
import pandas as pd

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt